In [1]:
library(dplyr)
library(ggplot2)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
source('/Users/ninhle/Desktop/Research/scPASU_pipeline_runs/SC296_scPASU_run/Scripts/cancercell/pa_site_ref_qc_plots/scPASU_plotting_functions.R')

out_dir<-'~/Desktop/Research/scPASU_pipeline_runs/Ureter10_scPASU_run/outputs/differentiation_stage_cellranger_peakcount/APA_peak_hexamer_check/'
tmp_dir<- paste0(out_dir,'tmp/')
fprefix <-'Ureter10_urothelial'


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: generics


Attaching package: ‘generics’


The following object is masked from ‘package:dplyr’:

    explain


The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union



Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:dplyr’:

    combine


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax

### PolyA Signal Search 

In [3]:
if (!dir.exists(out_dir)) {dir.create(out_dir, recursive = TRUE)}
if (!dir.exists(tmp_dir)) {dir.create(tmp_dir, recursive = TRUE)}

f<-'~/Desktop/Research/scPASU_pipeline_runs/Ureter10_scPASU_run/outputs/urothelial_final_peak_universe_updated.txt'

file<-fread(f,header=TRUE)
consensus_res <- read.delim(paste0(out_dir,'consensus_fractional_usage_each_stage.txt'))

## (A) For PR ##

# Please update column names if they differ in your table from the ones listed below

file2<-file[file$tu %in% consensus_res$TU,]
colnames(file2)[grepl('gene',colnames(file2))] <- 'gene_name'
colnames(file2)<-gsub('\\bchr\\b','peak_chr',colnames(file2))
colnames(file2)<-gsub('\\bstart\\b','peak_start',colnames(file2))

colnames(file2)<-gsub('\\bend\\b','peak_end',colnames(file2))
#colnames(file2)<-gsub('\\bstrand\\b','peak_strand',colnames(file2))

colnames(file2)<-gsub('\\bpr_chr\\b','chr',colnames(file2))
colnames(file2)<-gsub('\\bpr_start\\b','start',colnames(file2))
colnames(file2)<-gsub('\\bpr_end\\b','end',colnames(file2))
#colnames(file2)<-gsub('\\bpr_strand\\b','strand',colnames(file2))

gr2<-makeGRangesFromDataFrame(file2,ignore.strand = FALSE, keep.extra.columns = TRUE)

pr_hex<- findHex(gr2,us=40)

Generating flanks 40 bp upstream 
Pulling sequence data from hg38 
Locate most downstream hexamer if any 


### PolyA signal switching analysis

#### Lengthening

In [14]:
lgthn_utr3 <- read.delim('~/Desktop/Research/scPASU_pipeline_runs/Ureter10_scPASU_run/outputs/differentiation_stage_cellranger_peakcount/WUI_APA_genes_UTR3_lengthening_ranges.bed',
                         header = F)
colnames(lgthn_utr3) <- c('chr','pr_start','pr_end','gene','score','strand')
lgthn_utr3$gene <- gsub('_UTR3','',lgthn_utr3$gene)
lgthn_peaks <- file[file$gene %in% lgthn_utr3$gene,c('tu','gene','final_annotation','chr','start','end','pr_start','pr_end')]

lgthn_utr3 <- merge(lgthn_utr3, lgthn_peaks[,c("gene", "pr_start","final_annotation")], by = c("gene", "pr_start"), all.x = TRUE)
lgthn_utr3 <- merge(lgthn_utr3, lgthn_peaks[,c("gene", "pr_end","final_annotation")], by = c("gene", "pr_end"), all.x = TRUE)
lgthn_utr3$transition_peaks <- apply(lgthn_utr3, 1, function(x){
  sorted_peaks <- sort(c(x[7],x[8]),decreasing = F)
  transition_peaks <- paste0(sorted_peaks,collapse = '-')
  return(transition_peaks)
})

In [15]:
pr_hex_df <- as.data.frame(pr_hex)
hex_cols <- apply(lgthn_utr3, 1, function(x){
  sorted_peaks <- sort(c(x[7],x[8]),decreasing = F)
  old_hex <- pr_hex_df[pr_hex_df$final_annotation==sorted_peaks[1],'hex']
  new_hex <- pr_hex_df[pr_hex_df$final_annotation==sorted_peaks[2],'hex']
  return(c(old_hex = unfactor(old_hex), new_hex = unfactor(new_hex)))
}) %>% t() %>% as.data.frame() 
lgthn_utr3 <- cbind(lgthn_utr3,hex_cols)
lgthn_utr3 <- lgthn_utr3 %>% mutate(old_hex_type = ifelse(old_hex == "AATAAA", "Canonical", "NonCanonical"),
                                    new_hex_type = ifelse(new_hex == "AATAAA", "Canonical", "NonCanonical")) %>% 
  mutate(transition_hex_type = paste0(old_hex_type,'-',new_hex_type))
table(lgthn_utr3$transition_hex_type)
#>       Canonical-Canonical    Canonical-NonCanonical    NonCanonical-Canonical NonCanonical-NonCanonical 
#>       6                        27                        41                        59
# Supplementary Table S4
lgthn_utr3 <- lgthn_utr3[,c('gene','chr','pr_start','pr_end','strand',
              'final_annotation.x','final_annotation.y','transition_peaks',
              'old_hex','new_hex','old_hex_type','new_hex_type','transition_hex_type')] %>% 
  rename('final_annotation.x' = 'final_annotation.start',
         'final_annotation.y' = 'final_annotation.end')
# write.table(lgthn_utr3,paste0(out_dir,'pr_hexamers_transition_lengthening_ATbased.txt'),
#             quote = FALSE,sep = '\t', row.names = FALSE, col.names = TRUE)


      Canonical-Canonical    Canonical-NonCanonical    NonCanonical-Canonical 
                        6                        27                        41 
NonCanonical-NonCanonical 
                       59 

In [16]:
head(lgthn_utr3)

,gene,chr,pr_start,pr_end,strand,final_annotation.start,final_annotation.end,transition_peaks,old_hex,new_hex,old_hex_type,new_hex_type,transition_hex_type
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ADD1,chr4,2928225,2930200,+,TU8272:ADD1:P1,TU8272:ADD1:P2,TU8272:ADD1:P1-TU8272:ADD1:P2,NONE,GATAAA,NonCanonical,NonCanonical,NonCanonical-NonCanonical
2,AGPAT3,chr21,43984514,43986679,+,TU35682:AGPAT3:P1,TU35682:AGPAT3:P2,TU35682:AGPAT3:P1-TU35682:AGPAT3:P2,CATAAA,AATGAA,NonCanonical,NonCanonical,NonCanonical-NonCanonical
3,AK2,chr1,33009838,33013273,-,TU2205:AK2:P2,TU2205:AK2:P1,TU2205:AK2:P1-TU2205:AK2:P2,ATTAAA,AATAAA,NonCanonical,Canonical,NonCanonical-Canonical
4,AKAP9,chr7,92110314,92112890,+,TU14133:AKAP9:P1,TU14133:AKAP9:P2,TU14133:AKAP9:P1-TU14133:AKAP9:P2,AATAAA,AACAAA,Canonical,NonCanonical,Canonical-NonCanonical
5,ALDH16A1,chr19,49470455,49471140,+,TU33187:ALDH16A1:P1,TU33187:ALDH16A1:P2,TU33187:ALDH16A1:P1-TU33187:ALDH16A1:P2,AATAAA,ATTAAA,Canonical,NonCanonical,Canonical-NonCanonical
6,ANKIB1,chr7,92399164,92401776,+,TU14135:ANKIB1:P1,TU14135:ANKIB1:P2,TU14135:ANKIB1:P1-TU14135:ANKIB1:P2,GATAAA,AATAAA,NonCanonical,Canonical,NonCanonical-Canonical


#### Shortening

In [23]:
shrtn_utr3 <- read.delim('~/Desktop/Research/scPASU_pipeline_runs/Ureter10_scPASU_run/outputs/differentiation_stage_cellranger_peakcount/WUI_APA_genes_UTR3_shortening_ranges.bed',
                         header = F)
colnames(shrtn_utr3) <- c('chr','pr_start','pr_end','gene','score','strand')
shrtn_utr3$gene <- gsub('_UTR3','',shrtn_utr3$gene)
shrtn_peaks <- file[file$gene %in% shrtn_utr3$gene,c('tu','gene','final_annotation','chr','start','end','pr_start','pr_end')]

shrtn_utr3 <- merge(shrtn_utr3, shrtn_peaks[,c("gene", "pr_start","final_annotation")], by = c("gene", "pr_start"), all.x = TRUE)
shrtn_utr3 <- merge(shrtn_utr3, shrtn_peaks[,c("gene", "pr_end","final_annotation")], by = c("gene", "pr_end"), all.x = TRUE)
shrtn_utr3$transition_peaks <- apply(shrtn_utr3, 1, function(x){
  sorted_peaks <- sort(c(x[7],x[8]),decreasing = T)
  transition_peaks <- paste0(sorted_peaks,collapse = '-')
  return(transition_peaks)
})

In [ ]:
pr_hex_df <- as.data.frame(pr_hex)
hex_cols <- apply(shrtn_utr3, 1, function(x){
  sorted_peaks <- sort(c(x[7],x[8]),decreasing = T)
  old_hex <- pr_hex_df[pr_hex_df$final_annotation==sorted_peaks[1],'hex']
  new_hex <- pr_hex_df[pr_hex_df$final_annotation==sorted_peaks[2],'hex']
  return(c(old_hex = unfactor(old_hex), new_hex = unfactor(new_hex)))
}) %>% t() %>% as.data.frame() 
shrtn_utr3 <- cbind(shrtn_utr3,hex_cols)
shrtn_utr3 <- shrtn_utr3 %>% mutate(old_hex_type = ifelse(old_hex == "AATAAA", "Canonical", "NonCanonical"),
                                    new_hex_type = ifelse(new_hex == "AATAAA", "Canonical", "NonCanonical")) %>% 
  mutate(transition_hex_type = paste0(old_hex_type,'-',new_hex_type))
table(shrtn_utr3$transition_hex_type)
#>      Canonical-Canonical    Canonical-NonCanonical    NonCanonical-Canonical NonCanonical-NonCanonical 
#>      9                        49                        28                        78
shrtn_utr3 <- shrtn_utr3[,c('gene','chr','pr_start','pr_end','strand',
                            'final_annotation.x','final_annotation.y','transition_peaks',
                            'old_hex','new_hex','old_hex_type','new_hex_type','transition_hex_type')] %>% 
  rename('final_annotation.x' = 'final_annotation.start',
         'final_annotation.y' = 'final_annotation.end')
# Supplementary Table S4
# write.table(shrtn_utr3,paste0(out_dir,'pr_hexamers_transition_shortening_ATbased.txt'),
#             quote = FALSE,sep = '\t', row.names = FALSE, col.names = TRUE)


      Canonical-Canonical    Canonical-NonCanonical    NonCanonical-Canonical 
                        9                        49                        28 
NonCanonical-NonCanonical 
                       78 

In [25]:
head(shrtn_utr3)

,gene,chr,pr_start,pr_end,strand,final_annotation.start,final_annotation.end,transition_peaks,old_hex,new_hex,old_hex_type,new_hex_type,transition_hex_type
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ABHD2,chr15,89195506,89202542,+,TU26925:ABHD2:P1,TU26925:ABHD2:P2,TU26925:ABHD2:P2-TU26925:ABHD2:P1,AACAAA,ATTAAA,NonCanonical,NonCanonical,NonCanonical-NonCanonical
2,ACADM,chr1,75761222,75763767,+,TU705:ACADM:P1,TU705:ACADM:P2,TU705:ACADM:P2-TU705:ACADM:P1,ACTAAA,AATAGA,NonCanonical,NonCanonical,NonCanonical-NonCanonical
3,ACOT13,chr6,24701473,24705087,+,TU11870:ACOT13:P1,TU11870:ACOT13:P2,TU11870:ACOT13:P2-TU11870:ACOT13:P1,TTTAAA,AATAAA,NonCanonical,Canonical,NonCanonical-Canonical
4,ACOX1,chr17,75941320,75947053,-,TU31430:ACOX1:P3,TU31430:ACOX1:P1,TU31430:ACOX1:P3-TU31430:ACOX1:P1,AATAAA,AATGAA,Canonical,NonCanonical,Canonical-NonCanonical
5,ACP2,chr11,47235748,47239484,-,TU21377:ACP2:P2,TU21377:ACP2:P1,TU21377:ACP2:P2-TU21377:ACP2:P1,AATAAA,AATGAA,Canonical,NonCanonical,Canonical-NonCanonical
6,ADAMTSL4-AS2,chr1,150544551,150546686,-,TU2820:ADAMTSL4-AS2:P2,TU2820:ADAMTSL4-AS2:P1,TU2820:ADAMTSL4-AS2:P2-TU2820:ADAMTSL4-AS2:P1,AATACA,AATAAA,NonCanonical,Canonical,NonCanonical-Canonical
